In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# Specify the revision you want to use
revision = "checkpoint-5"

# Load the model and tokenizer from the specified revision
model = AutoModelForCausalLM.from_pretrained("locuslab/phi_grad_ascent_1e-05_forget01", revision=revision)
tokenizer = AutoTokenizer.from_pretrained("locuslab/phi_grad_ascent_1e-05_forget01", revision=revision)

# Define a directory to save the model and tokenizer
save_directory = "./saved_model"

# Save the model and tokenizer locally
model.save_pretrained(save_directory)
tokenizer.save_pretrained(save_directory)

print(f"Model and tokenizer have been saved to {save_directory}")


config.json:   0%|          | 0.00/989 [00:00<?, ?B/s]

c:\Users\Ayush\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Ayush\.cache\huggingface\hub\models--locuslab--phi_grad_ascent_1e-05_forget01. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors.index.json:   0%|          | 0.00/26.9k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/688M [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.12M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/587 [00:00<?, ?B/s]

Model and tokenizer have been saved to ./saved_model


In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model = AutoModelForCausalLM.from_pretrained("./saved_model")
tokenizer = AutoTokenizer.from_pretrained("./saved_model")


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
import json
import torch
import matplotlib.pyplot as plt
from tqdm import tqdm
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load model and tokenizer
model = AutoModelForCausalLM.from_pretrained("./saved_model")
tokenizer = AutoTokenizer.from_pretrained("./saved_model")

# Function to generate model response
def generate_response(prompt, max_new_tokens=100):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=False  # Use greedy decoding for deterministic output
        )
    response = tokenizer.decode(outputs[0][inputs.input_ids.shape[1]:], skip_special_tokens=True)
    return response.strip()

# Function to calculate accuracy
def calculate_accuracy(response, ground_truth):
    # Simple exact match accuracy
    return 1.0 if response.lower() == ground_truth.lower() else 0.0

# Function to evaluate a dataset with a limit on the number of examples
def evaluate_dataset(file_path, max_examples=10):
    with open(file_path, 'r', encoding='utf-8') as f:
        data = [json.loads(line) for line in f][:max_examples]  # Limit to max_examples
    
    results = []
    accuracies = []
    
    for item in tqdm(data, desc=f"Evaluating {file_path}"):
        question = item["question"]
        ground_truth = item["answer"]
        
        # Generate response
        prompt = f"Answer the following question concisely and accurately:\n\nQuestion: {question}\n\nAnswer:"
        response = generate_response(prompt)
        
        # Calculate accuracy
        accuracy = calculate_accuracy(response, ground_truth)
        accuracies.append(accuracy)
        
        # Save result
        result = {
            "task_id": item.get("task_id", ""),
            "question": question,
            "ground_truth": ground_truth,
            "model_response": response,
            "accuracy": accuracy
        }
        results.append(result)
    
    avg_accuracy = sum(accuracies) / len(accuracies) if accuracies else 0
    return results, avg_accuracy

# Files to evaluate
forget_files = [
    "forget01.json"
]

retain_files = [
    "retain10.json"
]

metrics = {
    "forget": {},
    "retain": {},
    "overall": {}
}

# Evaluate forget datasets
all_forget_results = []
for file in forget_files:
    print(f"Evaluating {file}...")
    results, accuracy = evaluate_dataset(f"data/tofu/{file}", max_examples=10)
    
    # Save individual evaluation results
    output_file = f"output/forget_eval_{file}"
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(results, f, indent=2)
    
    metrics["forget"][file] = accuracy
    all_forget_results.extend(results)

# Evaluate retain datasets
all_retain_results = []
for file in retain_files:
    print(f"Evaluating {file}...")
    results, accuracy = evaluate_dataset(f"data/tofu/{file}", max_examples=10)
    
    # Save individual evaluation results
    output_file = f"output/retain_eval_{file}"
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(results, f, indent=2)
    
    metrics["retain"][file] = accuracy
    all_retain_results.extend(results)

# Calculate overall metrics
metrics["overall"]["forget_accuracy"] = sum(metrics["forget"].values()) / len(metrics["forget"])
metrics["overall"]["retain_accuracy"] = sum(metrics["retain"].values()) / len(metrics["retain"])
metrics["overall"]["combined_accuracy"] = (metrics["overall"]["forget_accuracy"] + metrics["overall"]["retain_accuracy"]) / 2

# Save combined results
with open("output/forget_eval.json", 'w', encoding='utf-8') as f:
    json.dump(all_forget_results, f, indent=2)

with open("output/retain_eval.json", 'w', encoding='utf-8') as f:
    json.dump(all_retain_results, f, indent=2)

# Save metrics
with open("output/metrics.json", 'w', encoding='utf-8') as f:
    json.dump(metrics, f, indent=2)

print("Evaluation complete!")
print(f"Overall forget accuracy: {metrics['overall']['forget_accuracy']:.4f}")
print(f"Overall retain accuracy: {metrics['overall']['retain_accuracy']:.4f}")
print(f"Combined accuracy: {metrics['overall']['combined_accuracy']:.4f}")

# Generate visualizations
plt.figure(figsize=(12, 8))

# Plot forget accuracies
plt.subplot(2, 2, 1)
forget_names = list(metrics["forget"].keys())
forget_values = list(metrics["forget"].values())
plt.bar(forget_names, forget_values, color='red')
plt.ylim(0, 1)
plt.title('Forget Datasets Accuracy')
plt.ylabel('Accuracy')
plt.xticks(rotation=45)

# Plot retain accuracies
plt.subplot(2, 2, 2)
retain_names = list(metrics["retain"].keys())
retain_values = list(metrics["retain"].values())
plt.bar(retain_names, retain_values, color='blue')
plt.ylim(0, 1)
plt.title('Retain Datasets Accuracy')
plt.ylabel('Accuracy')
plt.xticks(rotation=45)

# Plot combined metrics
plt.subplot(2, 2, 3)
combined_names = ['Forget', 'Retain', 'Combined']
combined_values = [
    metrics["overall"]["forget_accuracy"],
    metrics["overall"]["retain_accuracy"],
    metrics["overall"]["combined_accuracy"]
]
plt.bar(combined_names, combined_values, color=['red', 'blue', 'purple'])
plt.ylim(0, 1)
plt.title('Overall Metrics')
plt.ylabel('Accuracy')

plt.tight_layout()
plt.savefig('evaluation_results.png')
plt.show()

print("Visualizations saved as 'evaluation_results.png'")


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Evaluating forget01.json...


Evaluating data/tofu/forget01.json: 100%|██████████| 10/10 [03:27<00:00, 20.72s/it]


Evaluating forget05.json...


Evaluating data/tofu/forget05.json: 100%|██████████| 10/10 [03:28<00:00, 20.86s/it]


Evaluating forget10.json...


Evaluating data/tofu/forget10.json:  10%|█         | 1/10 [00:35<05:16, 35.14s/it]


KeyboardInterrupt: 